[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/joshmaglione/CS102-Jupyter/main?labpath=.%2F10_Regression.ipynb) 

<a href="https://colab.research.google.com/github/joshmaglione/CS102-Jupyter/blob/main/10_Regression.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> 

[View on GitHub](https://github.com/joshmaglione/CS102-Jupyter/blob/main/10_Regression.ipynb)

# 10 : Regression

## Learning outcomes
By the end of this notebook, you should be able to:
- fit a simple regression model with `scikit-learn` and use it to make predictions
- visualize regression data and fitted lines using Matplotlib
- evaluate regression performance with appropriate summary measures (e.g., $R^2$ and error)

We'll continue learning about machine learning. We are still looking at *supervised learning*.

- What is the difference between *supervised* and *unsupervised* learning? 

- What are some examples of each?

In [ ]:
import numpy as np
import pandas as pd
from numpy import random as rng
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

We are still working with `scikit-learn` which can help us with several machine learning tasks.

![](imgs/scikitlearn1.png)

![](imgs/scikitlearn2.png)

## Linear regression

You probably already know what linear regression is if the name is unfamiliar.

It is used all the time in sciences.

Linear regression = finding the line of best fit.

![](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*N1-K-A43_98pYZ27fnupDA.jpeg)

We perform a linear regression when we want to analyze how some variables (usually one) *linearly* depend on other variables.

**Simplest:** we have data points in $\mathbb{R}^2$.
- We suspect the $y$-values depend, at least in part, on the $x$-values.
- We perform some linear algebra to get the line of best fit, usually written as
$$
	y = \beta x + \varepsilon
$$
- We calculate the $R^2$-value to see how well the line fits the data.
- If it's a decent fit, we can predict a reasonable range of $y$-values for a given $x$-value.

Let's build a simple example.

In [ ]:
x = 10 * rng.rand(50) 			# Random numbers between 0 and 10
noise = rng.randn(50)			# Random noise
y = 2 * x - 5 + noise 			# Noisy but near the line 2x - 5

In [ ]:
plt.xlabel("x")
plt.ylabel("y")
plt.grid()
_ = plt.scatter(x, y, color='blue', alpha=0.5, zorder=2)

There is a very *clear* trend in the data. We expect a high $r^2$-value.

We'll use `LinearRegression` to get the line of best fit in this simple example.

In [ ]:
# Fit the model
model = LinearRegression(fit_intercept=True)
model.fit(x[:, np.newaxis], y)
xfit = np.linspace(0, 10, 2)			# Only need 2 points to define a line
yfit = model.predict(xfit[:, np.newaxis])
beta = model.coef_[0]					# slope
epsilon = model.intercept_				# intercept
r2 = model.score(x[:, np.newaxis], y)	# r^2-value

# Plot the model
plt.scatter(x, y, color='blue', alpha=0.5, zorder=2)
plt.xlabel("x")
plt.ylabel("y")
plt.plot(xfit, yfit, color="red", label=f"$y = {beta:.2f}x {epsilon:.2f}$")
plt.text(0.1, 13, f"$R^2 = {r2:.2f}$", fontsize=12, color="green")
plt.title("Linear Regression")
plt.grid()
plt.legend()
plt.show()

We expected a very high $R^2$-value since we produced the data to have very little noise. 

**Try it yourself:** Scale the vector `noise` above and see how it impacts the $r^2$-value.

## Multidimensional data

Instead of looking for a line (of best fit), we will look for a hyperplane.

It's the same idea, except we have more "independent variables".

We will still have one dependent variable.

I gathered some data related to housing from 
- centralbank.ie
- cso.ie

#### Information on the data

The central bank tells us the interest rates for fixed rate mortgages for each quarter between 2015 and a little bit before 2020. 

The CSO tells us the number of new and existing houses sold in Ireland each month and their cumulative values. They also give the median house price for all of Ireland.

Putting this all together required some serious work. 

Below is the code to gather the data from the csv files. 

This is just for your information -- you do not need to understand this code -- though *you can*!

In [ ]:
# Optional to read
#   I am taking two separate data sets and combining them into one. I need to
#   clean the data up a bit in the process.
# 	I am sure there are ways to improve this. If you do so, tell me :)

# Data from https://www.centralbank.ie/statistics/data-and-analysis/statistical-data-in-csv
# Interest data
df_int = pd.read_csv(
	"data/Interest.csv", 
	encoding = "ISO-8859-2", 
	usecols=["Table B.3.1  Retail Interest Rates - Lending for House Purchase", 
		"Reporting Date", 
		"PDH - Fixed Rate - Over 3 years"
	],
	parse_dates=["Reporting Date"]
)
df_int = (df_int
	.query("`Table B.3.1  Retail Interest Rates - Lending for House Purchase` == 'Outstanding Amounts - Rates (%)'")
	.drop(columns=["Table B.3.1  Retail Interest Rates - Lending for House Purchase"])
	.rename(columns={
		"Reporting Date": "Date", 
		"PDH - Fixed Rate - Over 3 years": "Interest"
	})
)
def month_to_interest(date):
	rel_data = df_int.query(f"Date <= '{date}'")
	if rel_data.empty:
		return np.nan
	return rel_data.tail(1).Interest.values[0]

# Data from: https://data.cso.ie/table/HPM04
# So much housing data
from functools import reduce
df = pd.read_csv(
	"data/Household.csv",
	parse_dates=["Month"], 
	date_format='%Y %m'
)
df = df.rename(columns={"Statistic Label": "Label"})
dfs = [df.query(f"Label == '{x}'") for x in df.Label.unique()]
for i, df in enumerate(dfs):
	dfs[i] = (df
		.rename(columns={"VALUE": f"{df.Label.unique()[0]}"})
		.query("`Stamp Duty Event` == 'Executions'")
		.drop(columns=["Label", "UNIT", "Stamp Duty Event", "Eircode Output"])
	)
df = reduce(pd.merge, dfs)
df = (df
	.rename(columns={
		"Volume of Sales": "Volume", 
		"Median Price": "Median", 
		"Value of Sales": "Value"
	})
	.astype({"Volume": "i"})
	.query("`Type of Buyer` == 'All Buyer Types'")
	.drop(columns=["Type of Buyer", "Mean Sale Price"])
	.set_index("Month")
)
df_all = df.query("`Dwelling Status` == 'All Dwelling Statuses'")
df_new = df.query("`Dwelling Status` == 'New'")
df_exist = df.query("`Dwelling Status` == 'Existing'")
ser_int = pd.Series([month_to_interest(date) for date in df_new.index], index=df_new.index)
df = pd.DataFrame({
	"Volume_New": df_new.Volume,
	"Value_New": df_new.Value,
	"Volume_Existing": df_exist.Volume,
	"Value_Existing": df_exist.Value,
	"Interest": ser_int,
	"Median": df_all.Median
}).dropna()
# Celebrate

Alas, our data set!

In [ ]:
df.head()

#### About the data:

- Since interest rate is given by quarters, I just repeated them for the months within.
- Column meanings:
  - **Volume_New**: Number of new houses sold
  - **Value_New**: Approximate total value of new houses sold in millions of Euro
  - **Volume_Existsing**: Number of houses sold that were previous owned (now called "existing houses")
  - **Value_Existing**: Approximate total value of existing houses sold in millions of Euro
  - **Interest**: The interest rate for fixed mortgages for more than three years
  - **Meadian**: The median sale price for a house in Ireland 

#### Correlation data

Won't go into the details, but the correlation between two variables, say, $x$ and $y$ is a value between $-1$ and $1$.

Correlation measures *linear* dependency. For example $y$ *depends linearly* on $x$ if there are real numbers $\alpha$ and $\beta$ such that
$$
	y = \alpha x + \beta.
$$

In real life, variables may have many "factors" that influence their value. 

Loosely speaking, suppose 
$$
\begin{aligned}
	x &= a_1 + \cdots + a_m + c_1 + \cdots + c_{\ell}, \\
	y &= b_1 + \cdots + b_n + c_1 + \cdots + c_{\ell}.
\end{aligned}
$$

Here "$=$" means "is influenced by". 

The more factors in common (e.g. the $c_i$) the larger the correlation.

Lastly, $x$ and $y$ are *postiviely correlated* if an increase in one implies an increase in the other (everything else fixed).

And they are *negatively correlated* if an increase in one implies a decrease in the other. 

In [ ]:
import seaborn as sns
corrmat = df.corr()
sns.heatmap(corrmat, annot = True, square = True)

Now let's apply linear regression on the data. 

We'll look specifically at the two volume columns and the interest column.

Can we predict the Median column?

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Columns to use as dependent variables
COLS = df.columns[[0, 2, 4]]

X = df[COLS]
y = df['Median']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("MODEL DETAILS")
print("Variables:")
for i, col in enumerate(COLS.values):
	print(f"\tx_{i} = {col}")
print(f"\ty   = {y.name}")
print(f"Terms:\n\tepsilon = {model.intercept_:,.2f}")
for i, coef in enumerate(model.coef_):
	print(f"\tbeta_{i}  = {coef:,.2f}")
print(f"R^2-value: {model.score(X_test, y_test):.2f}")

With the above data, we get a linear function of the form:
$$
	y = \beta_0 x_0 + \beta_1 x_1 + \cdots + \beta_n x_n + \epsilon. 
$$

And the $r^2$-value tells us how well the above equation fits the data.

## The Bias-Variance Tradeoff

We seek balance.

![](https://ih1.redbubble.net/image.475188184.1134/flat,750x,075,f-pad,750x1000,f8f8f8.u3.jpg)

*This is one of the most important concepts in machine learning.*

In the next few examples, I will randomly generate some data.

Although it is not "real-world" data, the phenomena occur in the real-world. 

It can be hard sometimes to see the signs.

In [ ]:
# You can skip over this. 
# I am building data for the next figure.

# Some helper functions for the next few pictures
def make_data(N=30, err=0.8, rseed=1):
    # randomly sample the data
    rng = np.random.RandomState(rseed)
    X = rng.rand(N, 1) ** 2
    y = 10 - 1. / (X.ravel() + 0.1)
    if err > 0:
        y += err * rng.randn(N)
    return X, y

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

# fig0 ------------------------------------------------------------------------
X, y = make_data()
xfit = np.linspace(-0.1, 1.0, 1000)[:, None]
model1 = PolynomialRegression(1).fit(X, y)		# A high-bias model
model20 = PolynomialRegression(20).fit(X, y)	# A high variance model

fig0, ax0 = plt.subplots(1, 2, figsize=(16, 6))
fig0.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

ax0[0].scatter(X.ravel(), y, s=40, color='blue', alpha=0.5)
ax0[0].plot(xfit.ravel(), model1.predict(xfit), color='red')
ax0[0].axis([-0.1, 1.0, -2, 14])
ax0[0].set_title('High-bias model: Underfits the data', size=14)

ax0[1].scatter(X.ravel(), y, s=40, color='blue', alpha=0.5)
ax0[1].plot(xfit.ravel(), model20.predict(xfit), color='red')
ax0[1].axis([-0.1, 1.0, -2, 14])
ax0[1].set_title('High-variance model: Overfits the data', size=14)
plt.close(fig0)									# Shh, not yet!

# fig1 -------------------------------------------------------------------------
X2, y2 = make_data(10, rseed=42)

fig1, ax1 = plt.subplots(1, 2, figsize=(16, 6))
fig1.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

ax1[0].scatter(X.ravel(), y, s=40, c='blue', alpha=0.5)
ax1[0].plot(xfit.ravel(), model1.predict(xfit), color='red')
ax1[0].axis([-0.1, 1.0, -2, 14])
ax1[0].set_title('High-bias model: Underfits the data', size=14)
ax1[0].scatter(X2.ravel(), y2, s=40, c='green', alpha=0.5)
ax1[0].text(
	0.02, 
	0.98, 
	f"training score: $R^2$ = {model1.score(X, y):.2f}",
	ha='left', 
	va='top', 
	transform=ax1[0].transAxes, 
	size=14, 
	color='blue'
)
ax1[0].text(
	0.02, 
	0.91, 
	f"validation score: $R^2$ = {model1.score(X2, y2):.2f}",
	ha='left', 
	va='top', 
	transform=ax1[0].transAxes, 
	size=14, 
	color='green'
)

ax1[1].scatter(X.ravel(), y, s=40, c='blue', alpha=0.5)
ax1[1].plot(xfit.ravel(), model20.predict(xfit), color='red')
ax1[1].axis([-0.1, 1.0, -2, 14])
ax1[1].set_title('High-variance model: Overfits the data', size=14)
ax1[1].scatter(X2.ravel(), y2, s=40, c='green', alpha=0.5)
ax1[1].text(
	0.02, 
	0.98, 
	f"training score: $R^2$ = {model20.score(X, y):.2f}",
    ha='left', 
	va='top', 
	transform=ax1[1].transAxes, 
	size=14, 
	color='blue'
)
ax1[1].text(
	0.02, 
	0.91, 
	f"validation score: $R^2$ = {model20.score(X2, y2):,.2f}",
    ha='left', 
	va='top', 
	transform=ax1[1].transAxes, 
	size=14, 
	color='green'
)
plt.close(fig1)

# fig2 -------------------------------------------------------------------------
x = np.linspace(0, 1, 1000)
y1 = -(x - 0.5) ** 2
y2 = y1 - 0.33 + np.exp(x - 1)

fig2, ax2 = plt.subplots()
ax2.plot(x, y2, lw=10, alpha=0.5, color='blue')
ax2.plot(x, y1, lw=10, alpha=0.5, color='red')
ax2.text(0.15, 0.2, "training score", rotation=45, size=16, color='blue')
ax2.text(0.2, -0.05, "validation score", rotation=20, size=16, color='red')
ax2.text(
	0.02, 
	0.1, 
	'$\\longleftarrow$ High Bias', 
	size=18, 
	rotation=90, 
	va='center'
)
ax2.text(
	0.98, 
	0.1, 
	'$\\longleftarrow$ High Variance $\\longrightarrow$', 
	size=18, 
	rotation=90, 
	ha='right', 
	va='center'
)
ax2.text(
	0.48, 
	-0.12, 
	'Best$\\longrightarrow$\nModel', 
	size=18, 
	rotation=90, 
	va='center'
)
ax2.set_xlim(0, 1)
ax2.set_ylim(-0.3, 0.5)
ax2.set_xlabel('model complexity $\\longrightarrow$', size=14)
ax2.set_ylabel('model score $\\longrightarrow$', size=14)
ax2.xaxis.set_major_formatter(plt.NullFormatter())
ax2.yaxis.set_major_formatter(plt.NullFormatter())
ax2.set_title("Validation Curve Schematic", size=16)

plt.close(fig2)									# Shh, not yet!

# fig3 -------------------------------------------------------------------------
N = np.linspace(0, 1, 1000)
y1 = 0.75 + 0.2 * np.exp(-4 * N)
y2 = 0.7 - 0.6 * np.exp(-4 * N)

fig3, ax3 = plt.subplots()

ax3.plot(x, y1, lw=10, alpha=0.5, color='blue')
ax3.plot(x, y2, lw=10, alpha=0.5, color='red')
ax3.text(0.2, 0.88, "training score", rotation=-10, size=16, color='blue')
ax3.text(0.2, 0.5, "validation score", rotation=30, size=16, color='red')
ax3.text(
	0.98, 
	0.45, 
	'Good Fit $\\longrightarrow$', 
	size=18, 
	rotation=90, 
	ha='right', 
	va='center'
)
ax3.text(
	0.02, 
	0.57, 
	'$\\longleftarrow$ High Variance $\\longrightarrow$', 
	size=18, 
	rotation=90, 
	va='center'
)
ax3.set_xlim(0, 1)
ax3.set_ylim(0, 1)
ax3.set_xlabel('training set size $\\longrightarrow$', size=14)
ax3.set_ylabel('model score $\\longrightarrow$', size=14)
ax3.xaxis.set_major_formatter(plt.NullFormatter())
ax3.yaxis.set_major_formatter(plt.NullFormatter())

ax3.set_title("Learning Curve Schematic", size=16)
plt.close(fig3)

The balance we seek in life is between bias and variance.

Models have errors and these come from three main sources:
1. Variance,
2. Bias,
3. Noise.

**Variance** measures how much the model (e.g. a classifier) varies as it trains on different data sets.

**Bias** measures how much the *expected* model differs from the *expected* output label.

**Noise** measures how much the output label differs on different data sets.

- We will not focus on noise here. 
- Bias and variance concern the model; noise is all about the data.

#### Example scenarios

(These are not exhaustive... these errors are more subtle.)

A model has **high bias** when the model is too simple. 
- Suppose the data is inherently nonlinear but the model is a linear regression. No amount of extra training data will fix this problem.

A model has **high variance** when the model is overly complicated.
- Overfitting exemplifies this. A degree $100$ polynomial has an $r^2$ of $0.999997$, but on new data it performs very poorly.

Here's a great illustration:

![](https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/images/bias_variance/bullseye.png)

In data, this might look like:

In [ ]:
fig0

#### A little bit about $R^2$-values

- Used in regression models to measure how well the function fits the data.
- $R^2$ is a value between $-\infty$ and $1$:
  - $R^2=1$ means the function perfectly models the data.
  - $R^2=0$ means the function simply gives the mean.
  - $R^2=-\infty$ means 😬☠️


- For high-bias models, the performance of the model on the validation set is similar to the performance on the training set.
- For high-variance models, the performance of the model on the validation set is far worse than the performance on the training set.

In [ ]:
fig1

We have a knob to control the complexity of our model -- the degree of the polynomial

When we do that and plot the results we get the following behavior.

In [ ]:
fig2

- The training score is everywhere higher than the validation score.
- For very low model complexity, the training data is underfit, which means that the model is a poor predictor both for the training data and for any previously unseen data.
- For very high model complexity, the training data is overfit, which means that the model predicts the training data very well, but fails for any previously unseen data.
- For some intermediate value, the validation curve has a maximum. This level of complexity indicates a **suitable trade-off between bias and variance.**

Another aspect of model complexity is *the optimal model will generally depend on the size of your training data.*

A plot of the training/validation score with respect to the size of the training set is known as a *learning curve*.
- A model of a given complexity will **overfit a small dataset**: 
  - training score is high, validation score is low.
- A model of a given complexity will **underfit a large dataset**: 
  - training score decreases, validation score increases.

In both cases, validation score generally won't be better than training score.

In [ ]:
fig3

In practice, models generally have more than one knob to turn. 

Thus, plots of validation and learning curves change from curves to surfaces. 

## Tuning hyperparameters

Regression is a class of algorithms for classification. 

The *parameters* would be the intercept and coefficients of the model. 
- These are somewhat internal to the particular model.

The *hyperparameters* are parameters that control the model or learning process. 
- These are parameters "outside" of the model. 
- Examples are 
  - the train-test split ratio
  - the degree of the polynomial for polynomial regression

Scikit learn gives us a way to tune the hyperparameters.

In [ ]:
X, y = make_data(40)
_ = plt.scatter(X.ravel(), y, c='blue', alpha=0.5)

In what follows, we build a "parameter grid".
- All the possible combinations of parameters to test
- The best set of parameters wins.

(There are ways to fine-tune even *this* process, but I'm refraining.)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

param_grid = [{
	'poly__degree': range(1, 10), 
	'ridge__alpha': [0.1, 1, 10, 100]
	}]
pipeline = Pipeline(steps=[
	('poly', PolynomialFeatures()), 
	('ridge', Ridge())
])

grid_search = GridSearchCV(pipeline, param_grid)

We are using a *ridge* regression rather than the simpler linear regression. 

It's roughly the same idea, but with a way to balance the results. 

The parameter 'alpha' controls this balance. 

[Read more about it](https://en.wikipedia.org/wiki/Ridge_regression) if you are interested.

In [ ]:
grid_search.fit(X, y)
grid_search.best_params_

In [ ]:
# Depict our model as a line
X_test = np.linspace(-0.1, 1.1, 500)[:, None]

model = grid_search.best_estimator_

plt.scatter(X.ravel(), y, c='blue', alpha=0.5)
lim = plt.axis()
plt.axis(lim)
y_test = model.fit(X, y).predict(X_test)
plt.text(0.02, 0.98, f"$R^2$ = {model.score(X, y):.2f}", ha='left', va='top', transform=plt.gca().transAxes, size=12)
plt.text(0.2, 0.1, f"Equation: $y = {model.named_steps['ridge'].coef_[-1]:.2f}x^{grid_search.best_params_['poly__degree']} + $ lower degree terms", ha='left', va='top', transform=plt.gca().transAxes, size=12)
_ = plt.plot(X_test.ravel(), y_test, c='red')

## Exercises

1. Scikit learn comes with a dataset for [California house prices](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html). Try building a regression model on this data:

In [ ]:
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()